In [24]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from itertools import islice
from urllib.parse import urlparse


In [45]:
# Load filtered an streamed FineWeb 2 dataset
dataset = load_dataset("HuggingFaceFW/fineweb-2", split="train", data_dir="data/dan_Latn", streaming=True)

In [39]:
def get_batches(ds, batch_size):
    iterator = iter(ds)
    while True:
        batch = list(islice(iterator, batch_size))
        if not batch:
            break
        yield batch

In [40]:
gen = get_batches(dataset, 1000)

In [46]:
for i in gen:
    print(i)

In [27]:
def get_batches(iterator, batch_size):
    while True:
        batch = list(islice(iterator, batch_size))
        if not batch:
            break
        yield batch


def domain_distribution(df):
    if "domain" not in df.columns:
        raise ValueError("DataFrame does not contain a 'domain' column.")
    
    distribution = df["domain"].value_counts().sort_values(ascending=False)
    return distribution

In [36]:
Batch_Size = 1000
Batch_Iterations = 3
Remove_Domains = ["www.texaspoker.dk"]

# Initialize DataFrame to store all records
all_records = pd.DataFrame()
all_domains = []

batch_generator = get_batches(dataset, batch_size=Batch_Size)
for _ in range(Batch_Iterations):  # 3 batches = 3000 samples
    print(f"Batch {_} of {Batch_Iterations}")
    batch = next(batch_generator)

    # Filter and extract fields
    records = [{
        "text": example["text"],
        "minhash_cluster_size": example.get("minhash_cluster_size"),
        "url": example.get("url")
    } for example in batch]

    temp_Records = pd.DataFrame(records)
    temp_Records["domain"] = temp_Records["url"].apply(lambda url: urlparse(url).netloc)

    ### Possible Filtering Here
    filtered_Records = temp_Records[~temp_Records["domain"].isin(Remove_Domains)].reset_index(drop=True)
    temp_domains = filtered_Records["domain"].unique()
    ###

    all_records = pd.concat([all_records, filtered_Records], ignore_index=True)
    all_domains.extend(temp_domains)



    

Batch 0 of 3
Batch 1 of 3
Batch 2 of 3


In [38]:
print(domain_distribution(all_records))
print("Total Domains:",len(all_domains))

domain
www.tripadvisor.dk      228
da.wikipedia.org        105
www.trustpilot.co.uk    102
www.andersen.sdu.dk      75
www.bt.dk                75
                       ... 
www.nordkysten.tv         3
woman.dk                  3
www-01.ibm.com            3
www.degulesider.dk        3
www.r-r.dk                3
Name: count, Length: 623, dtype: int64
Total Domains: 1869


In [35]:
check_domain = "www.texaspoker.dk"

all_records[all_records["domain"] == check_domain]

,text,minhash_cluster_size,url,domain
998,Doyle Brunson har udgivet en ny blogpost hvor ...,37,http://www.texaspoker.dk/doyle-brunson-forsvar...,www.texaspoker.dk
1998,Doyle Brunson har udgivet en ny blogpost hvor ...,37,http://www.texaspoker.dk/doyle-brunson-forsvar...,www.texaspoker.dk
2998,Doyle Brunson har udgivet en ny blogpost hvor ...,37,http://www.texaspoker.dk/doyle-brunson-forsvar...,www.texaspoker.dk
